# Creating .txt input files to make the hst stamps for the plots

## Import section

In [31]:
from astropy.io.votable import is_votable, parse
from stage2 import *

from sys import exit

import multiprocessing
from multiprocessing import Manager

## Defining filenames and paths

In [32]:
pathdata  = "../outputs/SelectedGals_sep_by_cluster/"
nameBegin = "matching_fieldsGals_Cassata_and_Zurich_corrected_radius_"
listNumb  = [114, 23, 26, 28, 30, 32, 34, 51, 61, 79, 84, '84-N']
CGrNames  = [i+str(j) for i, j in zip(['CGr']*12, listNumb)]
fullNames = [pathdata + i + '/' + nameBegin + i + '.vot' for i in CGrNames]

In [33]:
for name in fullNames:
    voTag = is_VOtable(name)
    if voTag:
        table = parse(name)
        full  = table.get_first_table()
        
        print("Size of", name, "is", full.array.shape[0], "\n")
    else:
        exit("Exiting")

The file ../outputs/SelectedGals_sep_by_cluster/CGr114/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr114.vot is a VOtable, right ? True
Size of ../outputs/SelectedGals_sep_by_cluster/CGr114/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr114.vot is 20 

The file ../outputs/SelectedGals_sep_by_cluster/CGr23/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr23.vot is a VOtable, right ? True
Size of ../outputs/SelectedGals_sep_by_cluster/CGr23/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr23.vot is 22 

The file ../outputs/SelectedGals_sep_by_cluster/CGr26/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr26.vot is a VOtable, right ? True
Size of ../outputs/SelectedGals_sep_by_cluster/CGr26/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr26.vot is 20 

The file ../outputs/SelectedGals_sep_by_cluster/CGr28/matching_fieldsGals_Cassata_and_Zurich_corrected_radius_CGr28.vot is a VOtable, right ? True
Size of ../outputs/Selected

## Looping over files, keeping relevant data, renaming some columns and saving into a .txt file

In [43]:
def parsing_and_converting_files(fname, outname, overwrite=False, delimiter=None,
                                 form={'ID':'%3s', 'ID_LAIGLE':'%6s', 'z':'%.5f',
                                       'flag':'%1d', 'ra':'%3.6f', 'dec':'%3.6f', 'Iab':'%2.6f'},
                                 fields=['ID', 'ID_Laigle_16', 'Z_MUSE',
                                         'CONFID', 'RA', 'DEC', 'ip_MAG_ISO'],
                                 newFieldNames={'ID_Laigle_16':'ID_LAIGLE', 'Z_MUSE':'z', 'CONFID':'flag',
                                                'RA':'ra', 'DEC':'dec', 'ip_MAG_ISO':'Iab'}):
    
    catalog = Table(parse(fname).get_first_table().array)[fields]
    
    #renaming
    for field in newFieldNames.keys():
        catalog.rename_column(field, newFieldNames[field])
        
    #writing
    catalog.write(outname, format='ascii.fixed_width', formats=form, 
                  overwrite=overwrite, delimiter=None)

for file, cg in zip(fullNames, CGrNames):
    proc = multiprocessing.Process(name=cg, target=parsing_and_converting_files, 
                                   args=(file, '../data/hst/' + cg + '/' + 'HST_' + cg + '.txt'))
    proc.start()

Process CGr114:
Traceback (most recent call last):
  File "/home/wilfried/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/wilfried/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-43-7c7bb8e46a2a>", line 17, in parsing_and_converting_files
    overwrite=overwrite, delimiter=None)
  File "/home/wilfried/anaconda3/lib/python3.7/site-packages/astropy/table/table.py", line 2592, in write
    io_registry.write(self, *args, **kwargs)
  File "/home/wilfried/anaconda3/lib/python3.7/site-packages/astropy/io/registry.py", line 560, in write
    writer(data, *args, **kwargs)
  File "/home/wilfried/anaconda3/lib/python3.7/site-packages/astropy/io/ascii/connect.py", line 43, in io_write
    return write(table, filename, format=format, **kwargs)
  File "/home/wilfried/anaconda3/lib/python3.7/site-packages/astropy/io/ascii/ui.py", line 870, in write
    ra